# Mapping Green Zones

Create an interactive web site that promotes environmental justice and highlights the
characteristics of the protected municipal Green Zones in Minneapolis.

Link to Github: https://github.com/RwHendrickson/MappingGZ

## Packages needed for our project

In [14]:
### Import Libraries

# File manipulation

import os # For working with Operating System
from sys import platform # Diagnose operating system
import json # For working with Json files
import requests # Processing online requests


# Analysis

import numpy as np # For working with Arrays
import pandas as pd # Data Manipulation
import geopandas as gpd # Spatial Data Manipulation

# Visualization

import folium # Interactive Leaflet.js mapping 
from folium import features

import warnings
warnings.filterwarnings('ignore') # Ignores some warnings

In [15]:
### Definitions

cwd = os.getcwd() # Current Working Directory

# Forward or back slashs for filepaths? <- Not sure here. Only know Windows & Linux

if platform == "linux" or platform == "linux2":
    slash = '/'
elif platform == 'win32':
    slash = '\\'

## Load Data

In [80]:
# Load Data -  We'll have to convert all data to another projection... <- Maybe go back and do this in the cleaning notebooks
# Let's use WGS84 EPSG:4326...

# Get GeoJsons from GitHub

# Base URL

url = (
    "https://raw.githubusercontent.com/RwHendrickson/MappingGZ/main/Prototype/Notebooks/CleaningData"
)

# Minneapolis Boundary

mpls_boundary_path = f"{url}/Boundary/mpls_boundaryWGS84.geojson"

# GZ Boundaries

gz_boundary_path = f"{url}/GZ_Boundaries/gz_boundaryWGS84.geojson"

# Census tract boundaries

census_tracts = gpd.read_file('mpls_census_tracts.shp')

# Get bigger files from computer

cwd = os.getcwd()

mpls_aadt_path = cwd + slash + 'CleaningData' + slash + 'Traffic' + slash + 'mpls_aadtWGS84.geojson'
mpls_emissions_path = cwd + slash + 'CleaningData' + slash + 'PermittedEmissions' + slash + 'mpls_emissions.csv'
mpls_health_path = cwd + slash + 'CleaningData' + slash + 'Health' + slash + 'mpls_health_data.shp'

# Load files as geodataframes for focusing data

traffic = gpd.read_file(mpls_aadt_path)
emissions = pd.read_csv(mpls_emissions_path)
health = gpd.read_file(mpls_health_path)

emissions_geo = gpd.GeoDataFrame(emissions, geometry = gpd.points_from_xy(emissions.LONGITUDE, emissions.LATITUDE, crs = 'EPSG:4326'))

In [1]:
'''
#Taylor's workaround

traffic = gpd.read_file('mpls_aadtWGS84.geojson')
emissions = pd.read_csv('mpls_emissions.csv')
health = gpd.read_file('mpls_health_data.shp')
emissions_geo = gpd.GeoDataFrame(emissions, geometry = gpd.points_from_xy(emissions.LONGITUDE, emissions.LATITUDE, crs = 'EPSG:4326'))
'''
#since I am using latis, the files are not pulling directly from my computer so I needed a workaround to access the files

"\n#Taylor's workaround\n\ntraffic = gpd.read_file('mpls_aadtWGS84.geojson')\nemissions = pd.read_csv('mpls_emissions.csv')\nhealth = gpd.read_file('mpls_health_data.shp')\nemissions_geo = gpd.GeoDataFrame(emissions, geometry = gpd.points_from_xy(emissions.LONGITUDE, emissions.LATITUDE, crs = 'EPSG:4326'))\n"

In [109]:
# Focus Data

## Traffic

aadt = traffic[['SEQUENCE_N', 'ROUTE_LABE', 'CURRENT_VO', 'geometry']]

# Emissions

emissions_2020 = emissions_geo[emissions_geo['YEAR']==2020][['FACILITY_NAME', 'INDUSTRY_TYPE', 'NAICS_CODE', 'POLLUTANT', 'LBS_EMITTED','geometry']]
vocs = emissions_2020[emissions_2020['POLLUTANT'] == 'Volatile Organic Compounds']
# pm = emissions_2020[emissions_2020['POLLUTANT'] == 'PM Primary']

# Health
health = health[['TractFIPS', 'TotalPopul', 'ACCESS2_Cr', 'ARTHRITIS_', 'BINGE_Crud', 'BPMED_Crud', 'CANCER_Cru', 'CASTHMA_Cr', 'CERVICAL_C', 'CHD_CrudeP', 'CHECKUP_Cr','CHOLSCREEN', 'COLON_SCRE', 'COPD_Crude', 'COREM_Crud', 'COREW_Crud', 'CSMOKING_C', 'DENTAL_Cru', 'DEPRESSION', 'DIABETES_C', 'GHLTH_Crud', 'HIGHCHOL_C', 'KIDNEY_Cru', 'LPA_CrudeP', 'MAMMOUSE_C', 'MHLTH_Crud', 'OBESITY_Cr', 'PHLTH_Crud', 'SLEEP_Crud', 'STROKE_Cru', 'TEETHLOST_', 'geometry']]   

## Define Visualizations

### Annual Average Daily Traffic

In [83]:
### AADT style

def style_aadt(feature):
    '''Set Thickness/color of streets to be proportional to traffic volume'''
    
    volume = feature['properties']['CURRENT_VO']
    
    
    # This is not the way to add a popup
    popup = f"Road: {feature['properties']['ROUTE_LABE']}\n Current Average Vehicles per Day: {feature['properties']['CURRENT_VO']}" 
    
    if volume < 1773: # Low volume
        return {
        "weight": 0.5,
        "color": "#848484",
        'popup': popup
    }
    elif volume < 5378: # Low-mid volume
        return {
        "weight": 1,
        "color": "#936d6d",
        'popup': popup
    }
    elif volume < 16308: # Mid volume
        return {
        "weight": 2,
        "color": "#a94646",
        'popup': popup
    }
    elif volume < 100000: # High Volume
        return {
        "weight": 5,
        "color": "#f90707",
        'popup': popup
    }
    else: # Very High Volume Potentially could use another break here
        return {
        "weight": 10,
        "color": "#090707",
        'popup': popup
    }
    

### Permitted Volatile Organic Compound Emissions

In [84]:
    
# Plot the facilities' VOC emission
    
def plot_voc(point):
    '''plot circle of facilities related to amount of vocs emitted'''    
    
    lbs = point.LBS_EMITTED 
    
    if lbs < 25: # Low volume
        weight = 1,
        color = '#848484'
    elif lbs < 100: # Low-mid volume
        weight = 3,
        color = '#936d6d'
    elif lbs < 1000: # Mid volume
        weight = 5,
        color = '#a94646'
    elif lbs < 10000: # High Volume
        weight = 10,
        color = '#f90707'
    else: # Very High Volume
        weight = 15,
        color = '#090707'
        
    folium.Circle(location=[point.geometry.y,point.geometry.x],
                       weight = weight,
                       color = color,
                       fill = True,
                       popup = f'Facility: {point.FACILITY_NAME}\n Lbs of VOCs Emitted in 2020: {lbs}\n Industry Type: {point.INDUSTRY_TYPE}\n NAICS code: {point.NAICS_CODE}'
                       ).add_to(voc_feature_group)

## Add to the map!

In [111]:
# Make a basic folium map

m = folium.Map(location=[44.986656, -93.258133],
               zoom_start=12,
               tiles = 'cartodbpositron')

# Add boundary

folium.GeoJson(mpls_boundary_path,
               style_function = lambda feature: {'color':'#000000', # Border color
                                                 'fillOpacity' : '0',
                                                 'weight': '0.75'}, # Fill transparency
               name="Minneapolis Boundary", # Layer's Name
               control = False # Can you toggle in layer control?
              ).add_to(m)

# Add GZ boundaries

folium.GeoJson(gz_boundary_path, 
               style_function = lambda feature: {'fillColor':'#84E884',
                                 'color':'#003700',
                                 'fillOpacity': '0.5',
                                 'weight': '0'}, # Thickness of lines
               name = "Green Zone Boundaries",
               control = False).add_to(m)

# Add census tract bounbaries
folium.GeoJson(data=census_tracts["geometry"],
              style_function = lambda feature: {'color':'#000000', # Border color
                                                'fillOpacity' : '0',
                                                'weight': '0.75'}, # Fill transparency
              name = 'Census Tract Boundaries',
              control = False
              ).add_to(m)

# Add traffic

folium.GeoJson(aadt,
               style_function = lambda feature: style_aadt(feature), 
               name = 'Average Annual Daily Traffic',
               show = False # Does this appear in default of map
              ).add_to(m)

# Add Volatile Organic Compounds

voc_feature_group = folium.FeatureGroup(name = 'VOC Emitters',
                                           show = False) # Create Feature Group
vocs.apply(plot_voc, axis = 1) # Add to group
m.add_child(voc_feature_group)

# Add PLACES Health Data

folium.GeoJson(data=health["geometry"],
                 name = 'PLACES Health Data',
                #icon = folium.Icon(color='lightgray', icon='home') ,
                 popup = (f'Prevalence of the following health or health-related conditions of people 18 years or older {health["KIDNEY_Cru"]}'),
                 ).add_to(m)
    ## the pop up is not currently working, but for our final I hope to have all of the health data showing in a pop up that is connected to the census tract polygon, not the centroids of the tracts

# Turn on layer control

m.add_child(folium.map.LayerControl())

In [112]:
# Save Map

m.save(os.path.join('.', 'MVP.html'))

## Data Demo

This notebook represents our data demo because we have been able to add more data to our map than in our MVP and have started to think about the visualization for our final product. For additions in visualization, we have added toggle layers to toggle between the data sets as well as the boundaries of the Green Zones themselves. Because the census and health data are both by census tract, we have added the census tract boundaries to the map and added another layer of PLACES health data. Moving forward, we hope to combine the health & census data and have all of it appear in pop-ups on the map according to the actual polygons of the tracts rather than the centroids within the tract which the map is currently showing. In our final we will also be adding the "What's in my neighborhood" data which will be shown as pop-up points as well.